# 06_report_export_notify


In [0]:
# 06_report_export_notify
from pathlib import Path
from datetime import datetime
out_base = '/dbfs/FileStore/kyc/reports'
Path(out_base).mkdir(parents=True, exist_ok=True)

try:
    q = spark.table('finance.kyc_gold.report_export_queue').filter("status='QUEUED'")
except Exception as e:
    print('No queue', e); q = []

for r in q.collect():
    out = f"{out_base}/{r.report_name}_{datetime.utcnow().strftime('%Y%m%d_%H%M%S')}"
    try:
        df = spark.table(r.view_full_name)
        df.coalesce(1).write.mode('overwrite').option('header',True).csv(out)
        spark.sql(f"UPDATE finance.kyc_gold.report_export_queue SET status='DONE', finished_at = current_timestamp(), export_path='{out}' WHERE id = '{r.id}'")
        print('Exported', r.report_name, '->', out)
    except Exception as e:
        spark.sql(f"UPDATE finance.kyc_gold.report_export_queue SET status='FAILED' WHERE id = '{r.id}'")
        print('Export failed', r.id, e)
print('Done')